# Get Semantic Model synonyms to enrich AI Notes for Data Agents

In the current state of Fabric Data Agents (early June 2025), any enhancements done to a Semantic Model (like adding synonyms, descriptions etc.) are not respected by Data Agents. As a result, the Data Agent may start generating implicit measures not respecting any of your model enhancements. 

This Notebook helps you with the following tasks: 
- Collect the synonyms from an existing Semantic Model (only the ones added by a user)
- Create a text string containing all these synonyms
- Create a Fabric Data Agent
- Connect the Semantic Model as a source to the Data Agent
- Adds the text string with synonyms as AI notes to the Data Agent

What's left for you to do:
- Open the Data Agent
- Select the tables you want to be used by the Data Agent
- Take it for some test rides! 

All below solutions are based on the following open source libraries: 
- Semantic Link Labs: [GitHub](https://github.com/microsoft/semantic-link-labs)
- Fabric Data Agent SDK: [Docs](https://learn.microsoft.com/en-us/fabric/data-science/fabric-data-agent-sdk) [GitHub](https://github.com/microsoft/fabric-samples/tree/main/docs-samples/data-science/Fabric-Data-Agent-SDK/Samples)


In [2]:
# Import Sempy + labs
%pip install semantic-link-labs

# Installing Fabric Data Agent SDK
%pip install fabric-data-agent-sdk

from fabric.dataagent.client import (
    FabricDataAgentManagement,
    create_data_agent,
) 
import sempy_labs as labs
import sempy.fabric as fabric

StatementMeta(, 65546afc-b61d-48cc-ad51-eb7114dffd02, 10, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.5/723.5 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 213.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.9.3
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-155d6b77-9476-4f2d-9a6d-cd06e264ee2e
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# specify existing semantic model name and location
semanticmodel_name = "AW2020"
workspace_name = "AI Skills / Data Agent"

# Specify the desired name for the data agent (note that spaces are not allowed)
data_agent_name = "test_automation"

StatementMeta(, 65546afc-b61d-48cc-ad51-eb7114dffd02, 12, Finished, Available, Finished)

## Synonym properties
#### Source
One of the properties is Source. In case a field (measure, column or any other object) is renamed in a Power BI visual, the column [Source] will return the value "PowerBI.VisualColumnRename". 

#### State of synonyms
Synonyms can have various states. 
- Suggested
- Generated 
- None

Algorithms behind the Semantic Model can suggest synonyms, just like you in the Natural Language query capability of Power BI. This behavior is described in [this documentation](https://learn.microsoft.com/en-us/power-bi/natural-language/q-and-a-tooling-intro#field-synonyms). 

#### Last Modified
As suggestions are not always accurate, we aim to only find the user added synonyms. All synonyms that are not entered by a user, do not have a [Last Modified] datetime value set. Therefore, we filter the column [Last Modified] to only show the records that have a value. User added synonyms have the [State] = None. 

In [4]:
# Get all synynyms in the semantic model
allsynonyms = labs.list_synonyms(dataset = semanticmodel_name, workspace= workspace_name)

# By default the synonym field is populated with the object name. Therefore, filtering down to only relevant ones
# filter out all rows where the object name is the same as the synonym
relevantsynonyms = allsynonyms["IsExactMatch"] = allsynonyms["Object Name"].str.lower() == allsynonyms["Synonym"].str.lower()

StatementMeta(, 65546afc-b61d-48cc-ad51-eb7114dffd02, 13, Finished, Available, Finished)

In [26]:
# Get all synonyms which are explicitly user modified 
useraddedsynonyms = allsynonyms[allsynonyms["Last Modified"] != "None"]

# Number of relevant synonyms found
count = useraddedsynonyms["Synonym"].count()
print(f"Number of user modified synonyms found: {count}")

# Print synonyms to screen
useraddedsynonyms.head()

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 58, Finished, Available, Finished)

Number of user modified synonyms found: 3


,Culture Name,Table Name,Object Name,Object Type,Synonym,Type,State,Source,Weight,Last Modified,IsExactMatch
1,en-US,FactInternetSales,# Orders,Measure,number of orders,None,None,None,0.0,2025-06-03T16:19:29.083Z,False
274,en-US,FactInternetSales,$ Sales Amount,Measure,revenue,None,None,None,0.0,2025-06-03T15:14:54.033Z,False
275,en-US,FactInternetSales,$ Sales Amount,Measure,total sales amount,None,None,None,0.0,2025-06-03T16:19:36.443Z,False


In [36]:
# Start building the result text
result_text = f"The following synonyms are defined for Semantic Model {semanticmodel_name}:\n"

# Collect object names and synonyms
object_names = useraddedsynonyms["Object Name"].head()
synonyms = useraddedsynonyms["Synonym"].head()

# Append each line to the result text
for obj, syn in zip(object_names, synonyms):
    result_text += f'"{obj}" = "{syn}"\n'

# Print or use the result_text for reference
print(result_text)


StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 68, Finished, Available, Finished)

The following synonyms are defined for Semantic Model AW2020:
"# Orders" = "number of orders"
"$ Sales Amount" = "revenue"
"$ Sales Amount" = "total sales amount"



# Setting up Data Agent
Now we have all the information about the Semantic Model, let's setup a Data Agent and populate the instructions with the right information we collected above. 

In [28]:
# create Data Agent
data_agent = create_data_agent(data_agent_name)

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 60, Finished, Available, Finished)

In [29]:
# by default the instructions and description for the data agent will be empty, we will update them later in the notebook
data_agent.get_configuration()

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 61, Finished, Available, Finished)

DataAgentConfiguration(instructions=None, user_description=None)

#### Attaching the Semantic Model as a source
In this step we're adding the Semantic Model we used earlier to collect Synonyms, as a source to the just created Data Agent.

In [30]:
# Add a semantic model as source to Data Agent
# Note that the semantic model name has been specified earlier in this notebook

data_agent.add_datasource(semanticmodel_name, type="semanticmodel")
# datasource type could be: lakehouse, kqldatabase, datawarehouse or semanticmodel

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 62, Finished, Available, Finished)

Datasource(69e3354c-b596-4c6b-b12f-0c3a3a2dfa81)

#### Update the AI Notes for the Data Agent
In this section we're populating the AI Notes for the Data Agent with the recently collected Synonyms from the Semantic Model. 

In [66]:
# Update Data Agent config to add instructions
data_agent.update_configuration(
    instructions=f"{result_text}",
    user_description=f"Data agent to assist users with insights from the Semantic Model {semanticmodel_name}",
)

# Get current configuration
data_agent.get_configuration()

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 98, Finished, Available, Finished)

DataAgentConfiguration(instructions='The following synonyms are defined for Semantic Model AW2020:\n"# Orders" = "number of orders"\n"$ Sales Amount" = "revenue"\n"$ Sales Amount" = "total sales amount"\n', user_description=None)

#### Showing all tables
As I cannot decide for you, which tables are seen as relevant for the Data Agent, below provides a list of tables and column which are part of the Semantic Model that we just linked to the Data Agent. Next, is for you to decide which model objects you want to include and add select them within the Data Agent. 

In [78]:
# Get first data source for Data Agent (assuming you only have one added, which has been added above)
datasource = data_agent.get_datasources()[0]

datasource.pretty_print()

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 110, Finished, Available, Finished)

 FactInternetSales
  | SalesOrderNumber
  | SalesOrderLineNumber
  | CustomerKey
  | ProductKey
  | OrderDateKey
  | DueDateKey
  | ShipDateKey
  | PromotionKey
  | CurrencyKey
  | SalesTerritoryKey
  | OrderQuantity
  | UnitPrice
  | ExtendedAmount
  | UnitPriceDiscountPct
  | DiscountAmount
  | ProductStandardCost
  | TotalProductCost
  | SalesAmount
  | TaxAmount
  | FreightAmount
  | CarrierTrackingNumber
  | CustomerPONumber
  | RevisionNumber
  | $ Sales Amount
  | # Orders
  | # Orders shipped
 DimCustomer
  | CustomerKey
  | GeographyKey
  | CustomerAlternateKey
  | Title
  | FirstName
  | MiddleName
  | LastName
  | NameStyle
  | BirthDate
  | MaritalStatus
  | Suffix
  | Gender
  | EmailAddress
  | YearlyIncome
  | TotalChildren
  | NumberChildrenAtHome
  | EnglishEducation
  | SpanishEducation
  | FrenchEducation
  | EnglishOccupation
  | SpanishOccupation
  | FrenchOccupation
  | HouseOwnerFlag
  | NumberCarsOwned
  | AddressLine1
  | AddressLine2
  | Phone
  | DateFirstPur

'AW2020'

For clarity, here is just a section with the tables only

In [74]:
import io
import sys

# Capture the output of pretty_print() as string
buffer = io.StringIO()
sys_stdout = sys.stdout  # Save original stdout
sys.stdout = buffer     # Redirect stdout to buffer

datasource.pretty_print()  # This will print into buffer

sys.stdout = sys_stdout    # Restore original stdout
structure_text = buffer.getvalue()

# Now parse the string to extract table names
table_names = []

for line in structure_text.splitlines():
    stripped = line.strip()
    if not stripped:
        continue
    if stripped.startswith("|") or stripped.startswith("'"):
        continue
    table_names.append(stripped)

print("Extracted table names:")
print(table_names)

StatementMeta(, 0c23fb9d-ca56-4e07-b1b2-8565bdf8b442, 106, Finished, Available, Finished)

Extracted table names:
['FactInternetSales', 'DimCustomer', 'DimDate', 'DimProduct', 'DimSalesTerritory', 'DimProductCategory', 'DimProductSubcategory']
